In [8]:
import pandas as pd
import numpy as np


In [9]:
df1 = pd.read_csv('u.data',sep='\t',names=['user_id','item_id','rating','timestamp'])
print(df1)
#make user_id x item_it array
#2d array --> 944 users x 1683 items
ratings = np.empty(shape=(944,1683))
ratings[:] = np.nan
#copy training data(given ratings) into 2d array
for i in range(df1.shape[0]):
  #print(i)
  temp = df1.iloc[[i]]
  ratings[temp['user_id'][i]][temp['item_id'][i]] = temp['rating'][i]
print('copied')

       user_id  item_id  rating  timestamp
0          196      242       3  881250949
1          186      302       3  891717742
2           22      377       1  878887116
3          244       51       2  880606923
4          166      346       1  886397596
...        ...      ...     ...        ...
99995      880      476       3  880175444
99996      716      204       5  879795543
99997      276     1090       1  874795795
99998       13      225       2  882399156
99999       12      203       3  879959583

[100000 rows x 4 columns]
copied


In [10]:
#convert categorical columns into numbers
train = pd.read_csv('u.user',sep='|',names=['user_id','age','sex','occupation','zip'])#names=['user_id','item_id','rating','timestamp'])
train['zip'].value_counts()

train['sex'] = train['sex'].astype('category')
train['sex'] = train['sex'].cat.codes
train['occupation'] = train['occupation'].astype('category').cat.codes
train['zip'] = train['zip'].astype('category').cat.codes
train

,user_id,age,sex,occupation,zip
0,1,24,1,19,622
1,2,53,0,13,689
2,3,23,1,20,270
3,4,24,1,19,331
4,5,33,0,13,133
...,...,...,...,...,...
938,939,26,0,18,282
939,940,32,1,0,31
940,941,20,1,18,743
941,942,48,0,10,566


In [11]:
#create 5 folds, train_fi ---> means train of ith fold
test_f1 = train.iloc[0:101]
train_f1 = train.iloc[101:]
train_f1.reset_index(drop=True,inplace=True)
#fold 2 
test_f2= train[0:91]
train_f2 = train[91:]
#fold 3 
test_f3= train[0:81]
train_f3 = train[81:]
#fold 4 
test_f4= train[0:71]
train_f4 = train[71:]
#fold 5 
test_f5= train[0:61]
train_f5 = train[61:]
train_f3.reset_index(drop=True,inplace=True)
train_f2.reset_index(drop=True,inplace=True)
train_f4.reset_index(drop=True,inplace=True)
train_f5.reset_index(drop=True,inplace=True)


In [ ]:
test_f1.shape

(101, 5)

In [ ]:
train_f3

,user_id,age,sex,occupation,zip
0,82,50,1,14,207
1,83,40,1,13,336
2,84,32,1,6,433
3,85,51,1,3,169
4,86,26,1,0,350
...,...,...,...,...,...
857,939,26,0,18,282
858,940,32,1,0,31
859,941,20,1,18,743
860,942,48,0,10,566


In [ ]:
#kmeans with euclidean dist
#from sklearn doc
#-----------------------fold1---------------------
from sklearn.cluster import KMeans 

fitted_model = KMeans(n_clusters=20, random_state=0).fit(train_f1)

#train_cluster_index has the cluster numbers for the training data(users)
train_cluster_index = fitted_model.predict(train_f1)
#test_cluster_index has the cluster numbers for the test data(users)
test_cluster_index = fitted_model.predict(test_f1)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f1.shape[0]+1):
  #for each item J
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    #go through each training user 
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #found user at index k in our cluster
        #get user_id of user at index k 
        user_id = train_f1['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          #if our neighbour in our cluster has rated item J then use in finding average
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    #initialize predicted rating as 3 to minimize mae
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 1 : ',mae_sum/mae_cnt)


#same thing for fold 2 only thing is train_f1 is replaced by train_f2 and test_f1-->test_f2
#repeat for 5 folds
#-------------fold2--------------

fitted_model = KMeans(n_clusters=20, random_state=0).fit(train_f2)
train_cluster_index = fitted_model.predict(train_f2)
test_cluster_index = fitted_model.predict(test_f2)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
for i in range(1,test_f2.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f2['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 2 : ',mae_sum/mae_cnt)


#-----------------fold3--------------------

fitted_model = KMeans(n_clusters=20, random_state=0).fit(train_f3)
train_cluster_index = fitted_model.predict(train_f3)
test_cluster_index = fitted_model.predict(test_f3)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
for i in range(1,test_f3.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f3['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 3 : ',mae_sum/mae_cnt)


#------fold 4--------------

fitted_model = KMeans(n_clusters=20, random_state=0).fit(train_f4)
train_cluster_index = fitted_model.predict(train_f4)
test_cluster_index = fitted_model.predict(test_f4)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
for i in range(1,test_f4.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f4['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 4 : ',mae_sum/mae_cnt)


#----------F5---------

fitted_model = KMeans(n_clusters=20, random_state=0).fit(train_f5)
train_cluster_index = fitted_model.predict(train_f5)
test_cluster_index = fitted_model.predict(test_f5)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
for i in range(1,test_f5.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f5['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 5 : ',mae_sum/mae_cnt)

Mae Fold 1 :  0.8761332637780882
Mae Fold 2 :  0.8933536603169839
Mae Fold 3 :  0.887355867817912
Mae Fold 4 :  0.8837678811146912
Mae Fold 5 :  0.9038970862476037


In [ ]:
#mean shift clustering - uses rbf kernel with centroids
#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html#sklearn.cluster.MeanShift
from sklearn.cluster import MeanShift

#-------------fold1-----------------
#same as before but only fitted_model is different, here we use meanshift 
# rest all same
fitted_model = MeanShift(bandwidth=10).fit(train_f1)
train_cluster_index = fitted_model.predict(train_f1)
test_cluster_index = fitted_model.predict(test_f1)

#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f1.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f1['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 1 : ',mae_sum/mae_cnt)

#------------------------fold2------------
#all folds same as fold 1 so no need to explain
fitted_model = MeanShift(bandwidth=10).fit(train_f2)
train_cluster_index = fitted_model.predict(train_f2)
test_cluster_index = fitted_model.predict(test_f2)

#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f2.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f2['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 2 : ',mae_sum/mae_cnt)


#----------------fold3------------
#fold1
fitted_model = MeanShift(bandwidth=10).fit(train_f3)
train_cluster_index = fitted_model.predict(train_f3)
test_cluster_index = fitted_model.predict(test_f3)

#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f3.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f3['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 3 : ',mae_sum/mae_cnt)


#----------fold4----------------

fitted_model = MeanShift(bandwidth=10).fit(train_f4)
train_cluster_index = fitted_model.predict(train_f4)
test_cluster_index = fitted_model.predict(test_f4)

#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f4.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f4['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 4 : ',mae_sum/mae_cnt)


#fold5---------------------------

fitted_model = MeanShift(bandwidth=10).fit(train_f5)
train_cluster_index = fitted_model.predict(train_f5)
test_cluster_index = fitted_model.predict(test_f5)

#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f5.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f5['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 5 : ',mae_sum/mae_cnt)





Mae Fold 1 :  1.0579559805159662
Mae Fold 2 :  1.0662531276063387
Mae Fold 3 :  1.0668377321603129
Mae Fold 4 :  1.0624171746620725
Mae Fold 5 :  1.0572293716881151


In [12]:
# dropping user_id column and calling it train_fino denoting training data for ith fold but no user id
train_f1no = train_f1.drop(['user_id'],axis=1)
test_f1no = test_f1.drop(['user_id'],axis=1)
train_f2no = train_f2.drop(['user_id'],axis=1)
test_f2no = test_f2.drop(['user_id'],axis=1)
train_f3no = train_f3.drop(['user_id'],axis=1)
test_f3no = test_f3.drop(['user_id'],axis=1)
train_f4no = train_f4.drop(['user_id'],axis=1)
test_f4no = test_f4.drop(['user_id'],axis=1)
train_f5no = train_f5.drop(['user_id'],axis=1)
test_f5no = test_f5.drop(['user_id'],axis=1)


In [ ]:
#k means from scratch with kernel - rbf
#-----------fold1--------------------
from sklearn.metrics.pairwise import pairwise_kernels

#to store cluster numbers of training data
train_labels = np.empty((train_f1no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
#initialize the 10 centroids by choosing first 10 points
for i in range(10):
  cluster = []
  cluster.append(train_f1no.iloc[[i]]['age'][i])
  cluster.append(train_f1no.iloc[[i]]['sex'][i])
  cluster.append(train_f1no.iloc[[i]]['occupation'][i])
  cluster.append(train_f1no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
#iterate 1000 times
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity, gives us similarity matrix
  similarity_mat = pairwise_kernels(X=train_f1no,Y=np.array(cluster_centroids),metric='rbf')
  #similarity_mat[i][j] is similarity between ith datapoint in trainf1_no and j th cluster (cluster number j+1)
  for i in range(train_f1no.shape[0]):
    #for each user find most similar centroid
    max_sim = max(similarity_mat[i])#most similar centroid for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroids by taking centroid = avg of all columns of all vectors in current cluster
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f1no.shape[0]):
      if(train_labels[i]==(k+1)):
        #if this user is in kth cluster,use in average computation
        cnt+=1
        #add user I's data to calculate average
        sum_arr[0]+=(train_f1no['age'][i])
        sum_arr[1]+=(train_f1no['sex'][i])
        sum_arr[2]+=(train_f1no['occupation'][i])
        sum_arr[3]+=(train_f1no['zip'][i])
    if(cnt>0):
      for l in range(4):
        #changing centroid to the newly computed ones
        cluster_centroids[k][l] = sum_arr[l]/cnt

#now clusters and centroids computed,find out which cluster is closest to each test point
#assign new rating as avg rating of all users in closest cluster
similarity_mat = pairwise_kernels(X=test_f1no,Y=np.array(cluster_centroids),metric='rbf')
#similarity between test and centroids
mae_sum=0
mae_cnt=0
#for each test point
for i in range(test_f1no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f1['user_id'][i]][j])==True):
      #ignore unrated items
      continue
    avg_sum=0
    avg_cnt=0
    #calculate avg rating of users in best cluster (cluster closest to user I)
    for u in range(train_f1.shape[0]):
      if(train_labels[u] == cluster_num):
        #user u is user I's neighbour (same cluster)
        if(np.isnan(ratings[train_f1['user_id'][u]][j])==True):
          #ignore if unrated
          continue
        #calculate average
        avg_sum+=ratings[train_f1['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # calc mae
    mae_sum+=abs(ratings[test_f1['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 1 mae : ',mae_sum/mae_cnt)



#--------------fold2---------------------------
#same as above only different dataset
#train_f2no instead of train_f1no

train_labels = np.empty((train_f2no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f2no.iloc[[i]]['age'][i])
  cluster.append(train_f2no.iloc[[i]]['sex'][i])
  cluster.append(train_f2no.iloc[[i]]['occupation'][i])
  cluster.append(train_f2no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f2no,Y=np.array(cluster_centroids),metric='rbf')
  for i in range(train_f2no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f2no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f2no['age'][i])
        sum_arr[1]+=(train_f2no['sex'][i])
        sum_arr[2]+=(train_f2no['occupation'][i])
        sum_arr[3]+=(train_f2no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f2no = test_f2.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f2no,Y=np.array(cluster_centroids),metric='rbf')
mae_sum=0
mae_cnt=0
for i in range(test_f2no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f2['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f2.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f2['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f2['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f2['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 2 mae : ',mae_sum/mae_cnt)

#-----------fold3--------------------


train_f3no = train_f3.drop(['user_id'],axis=1)
train_labels = np.empty((train_f3no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f3no.iloc[[i]]['age'][i])
  cluster.append(train_f3no.iloc[[i]]['sex'][i])
  cluster.append(train_f3no.iloc[[i]]['occupation'][i])
  cluster.append(train_f3no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f3no,Y=np.array(cluster_centroids),metric='rbf')
  for i in range(train_f3no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f3no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f3no['age'][i])
        sum_arr[1]+=(train_f3no['sex'][i])
        sum_arr[2]+=(train_f3no['occupation'][i])
        sum_arr[3]+=(train_f3no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f3no = test_f3.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f3no,Y=np.array(cluster_centroids),metric='rbf')
mae_sum=0
mae_cnt=0
for i in range(test_f3no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f3['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f3.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f3['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f3['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f3['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 3 mae : ',mae_sum/mae_cnt)

#-----------fold4--------------------

train_f4no = train_f4.drop(['user_id'],axis=1)
train_labels = np.empty((train_f4no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f4no.iloc[[i]]['age'][i])
  cluster.append(train_f4no.iloc[[i]]['sex'][i])
  cluster.append(train_f4no.iloc[[i]]['occupation'][i])
  cluster.append(train_f4no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f4no,Y=np.array(cluster_centroids),metric='rbf')
  for i in range(train_f4no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f4no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f4no['age'][i])
        sum_arr[1]+=(train_f4no['sex'][i])
        sum_arr[2]+=(train_f4no['occupation'][i])
        sum_arr[3]+=(train_f4no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f4no = test_f4.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f4no,Y=np.array(cluster_centroids),metric='rbf')
mae_sum=0
mae_cnt=0
for i in range(test_f4no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f4['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f4.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f4['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f4['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f4['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 4 mae : ',mae_sum/mae_cnt)


#-----------fold5--------------------


train_f5no = train_f5.drop(['user_id'],axis=1)
train_labels = np.empty((train_f5no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f5no.iloc[[i]]['age'][i])
  cluster.append(train_f5no.iloc[[i]]['sex'][i])
  cluster.append(train_f5no.iloc[[i]]['occupation'][i])
  cluster.append(train_f5no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f5no,Y=np.array(cluster_centroids),metric='rbf')
  for i in range(train_f5no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f5no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f5no['age'][i])
        sum_arr[1]+=(train_f5no['sex'][i])
        sum_arr[2]+=(train_f5no['occupation'][i])
        sum_arr[3]+=(train_f5no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f5no = test_f5.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f5no,Y=np.array(cluster_centroids),metric='rbf')
mae_sum=0
mae_cnt=0
for i in range(test_f5no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f5['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f5.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f5['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f5['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f5['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 5 mae : ',mae_sum/mae_cnt)



    
    
      



[38, 1, 14, 261]
fold 1 mae :  0.8463183616942396
[32, 1, 5, 591]
fold 2 mae :  0.8491217433037755
[50, 1, 14, 207]
fold 3 mae :  0.864353685266646
[48, 0, 0, 527]
fold 4 mae :  0.8701899205934366
[27, 0, 0, 741]
fold 5 mae :  0.8741941092163947


In [ ]:
#kernel k means scratch - laplacian
#same as above but we change metric in kernel function to laplacian
#-----------fold1--------------------
train_f1no = train_f1.drop(['user_id'],axis=1)
train_labels = np.empty((train_f1no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f1no.iloc[[i]]['age'][i])
  cluster.append(train_f1no.iloc[[i]]['sex'][i])
  cluster.append(train_f1no.iloc[[i]]['occupation'][i])
  cluster.append(train_f1no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f1no,Y=np.array(cluster_centroids),metric='laplacian')
  for i in range(train_f1no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f1no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f1no['age'][i])
        sum_arr[1]+=(train_f1no['sex'][i])
        sum_arr[2]+=(train_f1no['occupation'][i])
        sum_arr[3]+=(train_f1no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f1no = test_f1.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f1no,Y=np.array(cluster_centroids),metric='laplacian')
mae_sum=0
mae_cnt=0
for i in range(test_f1no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f1['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f1.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f1['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f1['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f1['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 1 mae : ',mae_sum/mae_cnt)



#--------------fold2---------------------------


train_f2no = train_f2.drop(['user_id'],axis=1)
train_labels = np.empty((train_f2no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f2no.iloc[[i]]['age'][i])
  cluster.append(train_f2no.iloc[[i]]['sex'][i])
  cluster.append(train_f2no.iloc[[i]]['occupation'][i])
  cluster.append(train_f2no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f2no,Y=np.array(cluster_centroids),metric='laplacian')
  for i in range(train_f2no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f2no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f2no['age'][i])
        sum_arr[1]+=(train_f2no['sex'][i])
        sum_arr[2]+=(train_f2no['occupation'][i])
        sum_arr[3]+=(train_f2no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f2no = test_f2.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f2no,Y=np.array(cluster_centroids),metric='laplacian')
mae_sum=0
mae_cnt=0
for i in range(test_f2no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f2['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f2.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f2['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f2['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f2['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 2 mae : ',mae_sum/mae_cnt)

#-----------fold3--------------------


train_f3no = train_f3.drop(['user_id'],axis=1)
train_labels = np.empty((train_f3no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f3no.iloc[[i]]['age'][i])
  cluster.append(train_f3no.iloc[[i]]['sex'][i])
  cluster.append(train_f3no.iloc[[i]]['occupation'][i])
  cluster.append(train_f3no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f3no,Y=np.array(cluster_centroids),metric='laplacian')
  for i in range(train_f3no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f3no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f3no['age'][i])
        sum_arr[1]+=(train_f3no['sex'][i])
        sum_arr[2]+=(train_f3no['occupation'][i])
        sum_arr[3]+=(train_f3no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f3no = test_f3.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f3no,Y=np.array(cluster_centroids),metric='laplacian')
mae_sum=0
mae_cnt=0
for i in range(test_f3no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f3['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f3.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f3['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f3['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f3['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 3 mae : ',mae_sum/mae_cnt)

#-----------fold4--------------------

train_f4no = train_f4.drop(['user_id'],axis=1)
train_labels = np.empty((train_f4no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f4no.iloc[[i]]['age'][i])
  cluster.append(train_f4no.iloc[[i]]['sex'][i])
  cluster.append(train_f4no.iloc[[i]]['occupation'][i])
  cluster.append(train_f4no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f4no,Y=np.array(cluster_centroids),metric='laplacian')
  for i in range(train_f4no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f4no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f4no['age'][i])
        sum_arr[1]+=(train_f4no['sex'][i])
        sum_arr[2]+=(train_f4no['occupation'][i])
        sum_arr[3]+=(train_f4no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f4no = test_f4.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f4no,Y=np.array(cluster_centroids),metric='laplacian')
mae_sum=0
mae_cnt=0
for i in range(test_f4no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f4['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f4.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f4['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f4['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f4['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 4 mae : ',mae_sum/mae_cnt)


#-----------fold5--------------------


train_f5no = train_f5.drop(['user_id'],axis=1)
train_labels = np.empty((train_f5no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f5no.iloc[[i]]['age'][i])
  cluster.append(train_f5no.iloc[[i]]['sex'][i])
  cluster.append(train_f5no.iloc[[i]]['occupation'][i])
  cluster.append(train_f5no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f5no,Y=np.array(cluster_centroids),metric='laplacian')
  for i in range(train_f5no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f5no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f5no['age'][i])
        sum_arr[1]+=(train_f5no['sex'][i])
        sum_arr[2]+=(train_f5no['occupation'][i])
        sum_arr[3]+=(train_f5no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f5no = test_f5.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f5no,Y=np.array(cluster_centroids),metric='laplacian')
mae_sum=0
mae_cnt=0
for i in range(test_f5no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f5['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f5.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f5['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f5['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f5['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 5 mae : ',mae_sum/mae_cnt)


   
        


fold 1 mae :  0.8509463755286631
[32, 1, 5, 591]
fold 2 mae :  0.8522622501801788
[50, 1, 14, 207]
fold 3 mae :  0.8703823609234372
[48, 0, 0, 527]
fold 4 mae :  0.8718060828924221
[27, 0, 0, 741]
fold 5 mae :  0.8838848236364114


In [ ]:
#kernel k means scratch - chi2
#same as above but we change metric in kernel function to chi2
#-----------fold1--------------------
train_f1no = train_f1.drop(['user_id'],axis=1)
train_labels = np.empty((train_f1no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f1no.iloc[[i]]['age'][i])
  cluster.append(train_f1no.iloc[[i]]['sex'][i])
  cluster.append(train_f1no.iloc[[i]]['occupation'][i])
  cluster.append(train_f1no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f1no,Y=np.array(cluster_centroids),metric='chi2')
  for i in range(train_f1no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f1no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f1no['age'][i])
        sum_arr[1]+=(train_f1no['sex'][i])
        sum_arr[2]+=(train_f1no['occupation'][i])
        sum_arr[3]+=(train_f1no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f1no = test_f1.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f1no,Y=np.array(cluster_centroids),metric='chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f1no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f1['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f1.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f1['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f1['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f1['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 1 mae : ',mae_sum/mae_cnt)



#--------------fold2---------------------------


train_f2no = train_f2.drop(['user_id'],axis=1)
train_labels = np.empty((train_f2no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f2no.iloc[[i]]['age'][i])
  cluster.append(train_f2no.iloc[[i]]['sex'][i])
  cluster.append(train_f2no.iloc[[i]]['occupation'][i])
  cluster.append(train_f2no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f2no,Y=np.array(cluster_centroids),metric='chi2')
  for i in range(train_f2no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f2no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f2no['age'][i])
        sum_arr[1]+=(train_f2no['sex'][i])
        sum_arr[2]+=(train_f2no['occupation'][i])
        sum_arr[3]+=(train_f2no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f2no = test_f2.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f2no,Y=np.array(cluster_centroids),metric='chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f2no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f2['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f2.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f2['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f2['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f2['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 2 mae : ',mae_sum/mae_cnt)

#-----------fold3--------------------


train_f3no = train_f3.drop(['user_id'],axis=1)
train_labels = np.empty((train_f3no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f3no.iloc[[i]]['age'][i])
  cluster.append(train_f3no.iloc[[i]]['sex'][i])
  cluster.append(train_f3no.iloc[[i]]['occupation'][i])
  cluster.append(train_f3no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f3no,Y=np.array(cluster_centroids),metric='chi2')
  for i in range(train_f3no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f3no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f3no['age'][i])
        sum_arr[1]+=(train_f3no['sex'][i])
        sum_arr[2]+=(train_f3no['occupation'][i])
        sum_arr[3]+=(train_f3no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f3no = test_f3.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f3no,Y=np.array(cluster_centroids),metric='chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f3no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f3['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f3.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f3['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f3['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f3['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 3 mae : ',mae_sum/mae_cnt)

#-----------fold4--------------------

train_f4no = train_f4.drop(['user_id'],axis=1)
train_labels = np.empty((train_f4no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f4no.iloc[[i]]['age'][i])
  cluster.append(train_f4no.iloc[[i]]['sex'][i])
  cluster.append(train_f4no.iloc[[i]]['occupation'][i])
  cluster.append(train_f4no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f4no,Y=np.array(cluster_centroids),metric='chi2')
  for i in range(train_f4no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f4no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f4no['age'][i])
        sum_arr[1]+=(train_f4no['sex'][i])
        sum_arr[2]+=(train_f4no['occupation'][i])
        sum_arr[3]+=(train_f4no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f4no = test_f4.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f4no,Y=np.array(cluster_centroids),metric='chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f4no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f4['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f4.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f4['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f4['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f4['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 4 mae : ',mae_sum/mae_cnt)


#-----------fold5--------------------


train_f5no = train_f5.drop(['user_id'],axis=1)
train_labels = np.empty((train_f5no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f5no.iloc[[i]]['age'][i])
  cluster.append(train_f5no.iloc[[i]]['sex'][i])
  cluster.append(train_f5no.iloc[[i]]['occupation'][i])
  cluster.append(train_f5no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f5no,Y=np.array(cluster_centroids),metric='chi2')
  for i in range(train_f5no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f5no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f5no['age'][i])
        sum_arr[1]+=(train_f5no['sex'][i])
        sum_arr[2]+=(train_f5no['occupation'][i])
        sum_arr[3]+=(train_f5no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f5no = test_f5.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f5no,Y=np.array(cluster_centroids),metric='chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f5no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f5['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f5.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f5['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f5['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f5['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 5 mae : ',mae_sum/mae_cnt)


   
        


fold 1 mae :  0.8403788967637624
[32, 1, 5, 591]
fold 2 mae :  0.8550987035835975
[50, 1, 14, 207]
fold 3 mae :  0.8687279173753859
[48, 0, 0, 527]
fold 4 mae :  0.8627886987070171
[27, 0, 0, 741]
fold 5 mae :  0.8724494291596149


In [ ]:
#kernel k means scratch - additive_chi
#same as above but we change metric in kernel function to additive_chi2
#-----------fold1--------------------
train_f1no = train_f1.drop(['user_id'],axis=1)
train_labels = np.empty((train_f1no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f1no.iloc[[i]]['age'][i])
  cluster.append(train_f1no.iloc[[i]]['sex'][i])
  cluster.append(train_f1no.iloc[[i]]['occupation'][i])
  cluster.append(train_f1no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f1no,Y=np.array(cluster_centroids),metric='additive_chi2')
  for i in range(train_f1no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f1no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f1no['age'][i])
        sum_arr[1]+=(train_f1no['sex'][i])
        sum_arr[2]+=(train_f1no['occupation'][i])
        sum_arr[3]+=(train_f1no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f1no = test_f1.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f1no,Y=np.array(cluster_centroids),metric='additive_chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f1no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f1['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f1.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f1['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f1['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f1['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 1 mae : ',mae_sum/mae_cnt)



#--------------fold2---------------------------


train_f2no = train_f2.drop(['user_id'],axis=1)
train_labels = np.empty((train_f2no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f2no.iloc[[i]]['age'][i])
  cluster.append(train_f2no.iloc[[i]]['sex'][i])
  cluster.append(train_f2no.iloc[[i]]['occupation'][i])
  cluster.append(train_f2no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f2no,Y=np.array(cluster_centroids),metric='additive_chi2')
  for i in range(train_f2no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f2no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f2no['age'][i])
        sum_arr[1]+=(train_f2no['sex'][i])
        sum_arr[2]+=(train_f2no['occupation'][i])
        sum_arr[3]+=(train_f2no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f2no = test_f2.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f2no,Y=np.array(cluster_centroids),metric='additive_chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f2no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f2['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f2.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f2['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f2['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f2['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 2 mae : ',mae_sum/mae_cnt)

#-----------fold3--------------------


train_f3no = train_f3.drop(['user_id'],axis=1)
train_labels = np.empty((train_f3no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f3no.iloc[[i]]['age'][i])
  cluster.append(train_f3no.iloc[[i]]['sex'][i])
  cluster.append(train_f3no.iloc[[i]]['occupation'][i])
  cluster.append(train_f3no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f3no,Y=np.array(cluster_centroids),metric='additive_chi2')
  for i in range(train_f3no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f3no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f3no['age'][i])
        sum_arr[1]+=(train_f3no['sex'][i])
        sum_arr[2]+=(train_f3no['occupation'][i])
        sum_arr[3]+=(train_f3no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f3no = test_f3.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f3no,Y=np.array(cluster_centroids),metric='additive_chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f3no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f3['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f3.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f3['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f3['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f3['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 3 mae : ',mae_sum/mae_cnt)

#-----------fold4--------------------

train_f4no = train_f4.drop(['user_id'],axis=1)
train_labels = np.empty((train_f4no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f4no.iloc[[i]]['age'][i])
  cluster.append(train_f4no.iloc[[i]]['sex'][i])
  cluster.append(train_f4no.iloc[[i]]['occupation'][i])
  cluster.append(train_f4no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f4no,Y=np.array(cluster_centroids),metric='additive_chi2')
  for i in range(train_f4no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f4no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f4no['age'][i])
        sum_arr[1]+=(train_f4no['sex'][i])
        sum_arr[2]+=(train_f4no['occupation'][i])
        sum_arr[3]+=(train_f4no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f4no = test_f4.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f4no,Y=np.array(cluster_centroids),metric='additive_chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f4no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f4['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f4.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f4['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f4['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f4['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 4 mae : ',mae_sum/mae_cnt)


#-----------fold5--------------------


train_f5no = train_f5.drop(['user_id'],axis=1)
train_labels = np.empty((train_f5no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f5no.iloc[[i]]['age'][i])
  cluster.append(train_f5no.iloc[[i]]['sex'][i])
  cluster.append(train_f5no.iloc[[i]]['occupation'][i])
  cluster.append(train_f5no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f5no,Y=np.array(cluster_centroids),metric='additive_chi2')
  for i in range(train_f5no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f5no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f5no['age'][i])
        sum_arr[1]+=(train_f5no['sex'][i])
        sum_arr[2]+=(train_f5no['occupation'][i])
        sum_arr[3]+=(train_f5no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f5no = test_f5.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f5no,Y=np.array(cluster_centroids),metric='additive_chi2')
mae_sum=0
mae_cnt=0
for i in range(test_f5no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f5['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f5.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f5['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f5['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f5['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 5 mae : ',mae_sum/mae_cnt)


   
        


fold 1 mae :  0.8403788967637624
[32, 1, 5, 591]
fold 2 mae :  0.8550987035835975
[50, 1, 14, 207]
fold 3 mae :  0.8687279173753859
[48, 0, 0, 527]
fold 4 mae :  0.8627886987070171
[27, 0, 0, 741]
fold 5 mae :  0.8724494291596149


In [ ]:
#kernel k means scratch - cosine
#same as above but we change metric in kernel function to cosine
#-----------fold1--------------------
train_f1no = train_f1.drop(['user_id'],axis=1)
train_labels = np.empty((train_f1no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f1no.iloc[[i]]['age'][i])
  cluster.append(train_f1no.iloc[[i]]['sex'][i])
  cluster.append(train_f1no.iloc[[i]]['occupation'][i])
  cluster.append(train_f1no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f1no,Y=np.array(cluster_centroids),metric='cosine')
  for i in range(train_f1no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f1no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f1no['age'][i])
        sum_arr[1]+=(train_f1no['sex'][i])
        sum_arr[2]+=(train_f1no['occupation'][i])
        sum_arr[3]+=(train_f1no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f1no = test_f1.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f1no,Y=np.array(cluster_centroids),metric='cosine')
mae_sum=0
mae_cnt=0
for i in range(test_f1no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f1['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f1.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f1['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f1['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f1['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 1 mae : ',mae_sum/mae_cnt)



#--------------fold2---------------------------


train_f2no = train_f2.drop(['user_id'],axis=1)
train_labels = np.empty((train_f2no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f2no.iloc[[i]]['age'][i])
  cluster.append(train_f2no.iloc[[i]]['sex'][i])
  cluster.append(train_f2no.iloc[[i]]['occupation'][i])
  cluster.append(train_f2no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f2no,Y=np.array(cluster_centroids),metric='cosine')
  for i in range(train_f2no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f2no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f2no['age'][i])
        sum_arr[1]+=(train_f2no['sex'][i])
        sum_arr[2]+=(train_f2no['occupation'][i])
        sum_arr[3]+=(train_f2no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f2no = test_f2.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f2no,Y=np.array(cluster_centroids),metric='cosine')
mae_sum=0
mae_cnt=0
for i in range(test_f2no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f2['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f2.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f2['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f2['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f2['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 2 mae : ',mae_sum/mae_cnt)

#-----------fold3--------------------


train_f3no = train_f3.drop(['user_id'],axis=1)
train_labels = np.empty((train_f3no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f3no.iloc[[i]]['age'][i])
  cluster.append(train_f3no.iloc[[i]]['sex'][i])
  cluster.append(train_f3no.iloc[[i]]['occupation'][i])
  cluster.append(train_f3no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f3no,Y=np.array(cluster_centroids),metric='cosine')
  for i in range(train_f3no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f3no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f3no['age'][i])
        sum_arr[1]+=(train_f3no['sex'][i])
        sum_arr[2]+=(train_f3no['occupation'][i])
        sum_arr[3]+=(train_f3no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f3no = test_f3.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f3no,Y=np.array(cluster_centroids),metric='cosine')
mae_sum=0
mae_cnt=0
for i in range(test_f3no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f3['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f3.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f3['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f3['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f3['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 3 mae : ',mae_sum/mae_cnt)

#-----------fold4--------------------

train_f4no = train_f4.drop(['user_id'],axis=1)
train_labels = np.empty((train_f4no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f4no.iloc[[i]]['age'][i])
  cluster.append(train_f4no.iloc[[i]]['sex'][i])
  cluster.append(train_f4no.iloc[[i]]['occupation'][i])
  cluster.append(train_f4no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f4no,Y=np.array(cluster_centroids),metric='cosine')
  for i in range(train_f4no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f4no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f4no['age'][i])
        sum_arr[1]+=(train_f4no['sex'][i])
        sum_arr[2]+=(train_f4no['occupation'][i])
        sum_arr[3]+=(train_f4no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f4no = test_f4.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f4no,Y=np.array(cluster_centroids),metric='cosine')
mae_sum=0
mae_cnt=0
for i in range(test_f4no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f4['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f4.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f4['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f4['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f4['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 4 mae : ',mae_sum/mae_cnt)


#-----------fold5--------------------


train_f5no = train_f5.drop(['user_id'],axis=1)
train_labels = np.empty((train_f5no.shape[0],))
train_labels[:] = -1# labels will be 1 to 10
#10 centroids
cluster_centroids = []
for i in range(10):
  cluster = []
  cluster.append(train_f5no.iloc[[i]]['age'][i])
  cluster.append(train_f5no.iloc[[i]]['sex'][i])
  cluster.append(train_f5no.iloc[[i]]['occupation'][i])
  cluster.append(train_f5no.iloc[[i]]['zip'][i])
  cluster_centroids.append(cluster)

print(cluster_centroids[0])
for iter in range(1000):
  #assign clusters to train
  #calculate kernel similarity
  similarity_mat = pairwise_kernels(X=train_f5no,Y=np.array(cluster_centroids),metric='cosine')
  for i in range(train_f5no.shape[0]):
    max_sim = max(similarity_mat[i])#most similar cluster for user I
    for j in range(10):
      if(similarity_mat[i][j]==max_sim):
        #assign user I to cluster J
        train_labels[i] = j+1
        break
  #recompute centroid
  for k in range(10):
    sum_arr = [0]*4
    cnt=0
    for i in range(train_f5no.shape[0]):
      if(train_labels[i]==(k+1)):
        cnt+=1
        #choose user I as part of cluster (k+1)
        sum_arr[0]+=(train_f5no['age'][i])
        sum_arr[1]+=(train_f5no['sex'][i])
        sum_arr[2]+=(train_f5no['occupation'][i])
        sum_arr[3]+=(train_f5no['zip'][i])
    if(cnt>0):
      for l in range(4):
        cluster_centroids[k][l] = sum_arr[l]/cnt

test_f5no = test_f5.drop(['user_id'],axis=1)
similarity_mat = pairwise_kernels(X=test_f5no,Y=np.array(cluster_centroids),metric='cosine')
mae_sum=0
mae_cnt=0
for i in range(test_f5no.shape[0]):
  best_cluster = max(similarity_mat[i])
  cluster_num = 0
  for k in range(10):
    if(similarity_mat[i][k]==best_cluster):
      #user I assign to cluster k+1
      cluster_num = k+1
      break
  for j in range(1,1683):
    if(np.isnan(ratings[test_f5['user_id'][i]][j])==True):
      continue
    avg_sum=0
    avg_cnt=0
    for u in range(train_f5.shape[0]):
      if(train_labels[u] == cluster_num):
        if(np.isnan(ratings[train_f5['user_id'][u]][j])==True):
          continue
        avg_sum+=ratings[train_f5['user_id'][u]][j]
        avg_cnt+=1
    predicted_rating=3
    if(avg_cnt>0):
      predicted_rating = avg_sum/avg_cnt
    # print(predicted_rating)
    mae_sum+=abs(ratings[test_f5['user_id'][i]][j] - predicted_rating)
    mae_cnt+=1
print('fold 5 mae : ',mae_sum/mae_cnt)


   
        


fold 1 mae :  0.8504369334077472
[32, 1, 5, 591]
fold 2 mae :  0.8601844649429699
[50, 1, 14, 207]
fold 3 mae :  0.8724292656467519
[48, 0, 0, 527]
fold 4 mae :  0.8591190766469655
[27, 0, 0, 741]
fold 5 mae :  0.8868980625589301


In [ ]:
#agglomerative clustering manhattan
from sklearn.cluster import AgglomerativeClustering
#from sklearn doc
#----fold1---------
#same as normal k means , but different model
total_labels = AgglomerativeClustering(n_clusters=10,affinity='manhattan',linkage='average').fit_predict(train_f1no.append(test_f1no,ignore_index=True))

#stores cluster number of train and test respectivily
train_cluster_index = total_labels[0:train_f1no.shape[0]]
test_cluster_index = total_labels[train_f1no.shape[0]:]



#all this already explained
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f1.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f1['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 1 : ',mae_sum/mae_cnt)


#----fold2---------

total_labels = AgglomerativeClustering(n_clusters=10,affinity='manhattan',linkage='average').fit_predict(train_f2no.append(test_f2no,ignore_index=True))

train_cluster_index = total_labels[0:train_f2no.shape[0]]
test_cluster_index = total_labels[train_f2no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f2.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f2['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 2 : ',mae_sum/mae_cnt)

#----fold3---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='manhattan',linkage='average').fit_predict(train_f3no.append(test_f3no,ignore_index=True))

train_cluster_index = total_labels[0:train_f3no.shape[0]]
test_cluster_index = total_labels[train_f3no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f3.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f3['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 3 : ',mae_sum/mae_cnt)


#----fold4---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='manhattan',linkage='average').fit_predict(train_f4no.append(test_f4no,ignore_index=True))

train_cluster_index = total_labels[0:train_f4no.shape[0]]
test_cluster_index = total_labels[train_f4no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f4.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f4['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 4 : ',mae_sum/mae_cnt)

#----fold5---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='manhattan',linkage='average').fit_predict(train_f5no.append(test_f5no,ignore_index=True))

train_cluster_index = total_labels[0:train_f5no.shape[0]]
test_cluster_index = total_labels[train_f5no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f5.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f5['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 5 : ',mae_sum/mae_cnt)




Mae Fold 1 :  0.8436984276533
Mae Fold 2 :  0.8535159183588656
Mae Fold 3 :  0.8651153147893279
Mae Fold 4 :  0.8660461830369446
Mae Fold 5 :  0.8821614520752755


In [ ]:
#agglomerative clustering l1 norm
from sklearn.cluster import AgglomerativeClustering

#----fold1---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='l1',linkage='single').fit_predict(train_f1no.append(test_f1no,ignore_index=True))

train_cluster_index = total_labels[0:train_f1no.shape[0]]
test_cluster_index = total_labels[train_f1no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f1.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f1['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 1 : ',mae_sum/mae_cnt)


#----fold2---------

total_labels = AgglomerativeClustering(n_clusters=10,affinity='l1',linkage='single').fit_predict(train_f2no.append(test_f2no,ignore_index=True))

train_cluster_index = total_labels[0:train_f2no.shape[0]]
test_cluster_index = total_labels[train_f2no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f2.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f2['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 2 : ',mae_sum/mae_cnt)

#----fold3---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='l1',linkage='single').fit_predict(train_f3no.append(test_f3no,ignore_index=True))

train_cluster_index = total_labels[0:train_f3no.shape[0]]
test_cluster_index = total_labels[train_f3no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f3.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f3['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 3 : ',mae_sum/mae_cnt)


#----fold4---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='l1',linkage='single').fit_predict(train_f4no.append(test_f4no,ignore_index=True))

train_cluster_index = total_labels[0:train_f4no.shape[0]]
test_cluster_index = total_labels[train_f4no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f4.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f4['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 4 : ',mae_sum/mae_cnt)

#----fold5---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='l1',linkage='single').fit_predict(train_f5no.append(test_f5no,ignore_index=True))

train_cluster_index = total_labels[0:train_f5no.shape[0]]
test_cluster_index = total_labels[train_f5no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f5.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f5['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 5 : ',mae_sum/mae_cnt)




Mae Fold 1 :  0.8037710220168129
Mae Fold 2 :  0.8165433049084527
Mae Fold 3 :  0.8221479838133668
Mae Fold 4 :  0.8213111920938856
Mae Fold 5 :  0.8350883886079955


In [ ]:
#agglomerative clustering l2 norm
from sklearn.cluster import AgglomerativeClustering
#everything else same
#----fold1---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='l2',linkage='average').fit_predict(train_f1no.append(test_f1no,ignore_index=True))

train_cluster_index = total_labels[0:train_f1no.shape[0]]
test_cluster_index = total_labels[train_f1no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f1.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f1['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 1 : ',mae_sum/mae_cnt)


#----fold2---------

total_labels = AgglomerativeClustering(n_clusters=10,affinity='l2',linkage='average').fit_predict(train_f2no.append(test_f2no,ignore_index=True))

train_cluster_index = total_labels[0:train_f2no.shape[0]]
test_cluster_index = total_labels[train_f2no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f2.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f2['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 2 : ',mae_sum/mae_cnt)

#----fold3---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='l2',linkage='average').fit_predict(train_f3no.append(test_f3no,ignore_index=True))

train_cluster_index = total_labels[0:train_f3no.shape[0]]
test_cluster_index = total_labels[train_f3no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f3.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f3['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 3 : ',mae_sum/mae_cnt)


#----fold4---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='l2',linkage='average').fit_predict(train_f4no.append(test_f4no,ignore_index=True))

train_cluster_index = total_labels[0:train_f4no.shape[0]]
test_cluster_index = total_labels[train_f4no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f4.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f4['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 4 : ',mae_sum/mae_cnt)

#----fold5---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='l2',linkage='average').fit_predict(train_f5no.append(test_f5no,ignore_index=True))

train_cluster_index = total_labels[0:train_f5no.shape[0]]
test_cluster_index = total_labels[train_f5no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f5.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f5['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 5 : ',mae_sum/mae_cnt)




Mae Fold 1 :  0.8485258144924133
Mae Fold 2 :  0.8584694952464074
Mae Fold 3 :  0.8643877595451679
Mae Fold 4 :  0.865373654732918
Mae Fold 5 :  0.879350842522081


In [17]:
# optics with minkowski as metric

from sklearn.cluster import OPTICS

#-----fold1----------
total_labels = OPTICS(min_samples=5,metric='minkowski').fit_predict(train_f1no.append(test_f1no,ignore_index=True))
#rest same
train_cluster_index = total_labels[0:train_f1no.shape[0]]
test_cluster_index = total_labels[train_f1no.shape[0]:]


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f1.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f1['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 1 : ',mae_sum/mae_cnt)



#---------------fold2------------

#-----fold1----------
total_labels = OPTICS(min_samples=5,metric='minkowski').fit_predict(train_f2no.append(test_f2no,ignore_index=True))

train_cluster_index = total_labels[0:train_f2no.shape[0]]
test_cluster_index = total_labels[train_f2no.shape[0]:]


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f2.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f2['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 2 : ',mae_sum/mae_cnt)

#-------------fold3-------------


total_labels = OPTICS(min_samples=5,metric='minkowski').fit_predict(train_f3no.append(test_f3no,ignore_index=True))

train_cluster_index = total_labels[0:train_f3no.shape[0]]
test_cluster_index = total_labels[train_f3no.shape[0]:]


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f3.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f3['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 3 : ',mae_sum/mae_cnt)



#-----fold4----------
total_labels = OPTICS(min_samples=5,metric='minkowski').fit_predict(train_f4no.append(test_f4no,ignore_index=True))

train_cluster_index = total_labels[0:train_f4no.shape[0]]
test_cluster_index = total_labels[train_f4no.shape[0]:]


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f4.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f4['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 4 : ',mae_sum/mae_cnt)


#-----fold5----------
total_labels = OPTICS(min_samples=5,metric='minkowski').fit_predict(train_f5no.append(test_f5no,ignore_index=True))

train_cluster_index = total_labels[0:train_f5no.shape[0]]
test_cluster_index = total_labels[train_f5no.shape[0]:]


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f5.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f5['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 5 : ',mae_sum/mae_cnt)



Mae Fold 1 :  0.8591594161851399
Mae Fold 2 :  0.8782710961251048
Mae Fold 3 :  0.8832249867762779
Mae Fold 4 :  0.8780680009480718
Mae Fold 5 :  0.8897481410197657


In [18]:
#ward clustering with euclidean
from sklearn.cluster import AgglomerativeClustering

#----fold1---------
#just the model changes rest same
total_labels = AgglomerativeClustering(n_clusters=10,affinity='euclidean',linkage='ward').fit_predict(train_f1no.append(test_f1no,ignore_index=True))

train_cluster_index = total_labels[0:train_f1no.shape[0]]
test_cluster_index = total_labels[train_f1no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f1.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f1['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 1 : ',mae_sum/mae_cnt)


#----fold2---------

total_labels = AgglomerativeClustering(n_clusters=10,affinity='euclidean',linkage='ward').fit_predict(train_f2no.append(test_f2no,ignore_index=True))

train_cluster_index = total_labels[0:train_f2no.shape[0]]
test_cluster_index = total_labels[train_f2no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f2.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f2['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 2 : ',mae_sum/mae_cnt)

#----fold3---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='euclidean',linkage='ward').fit_predict(train_f3no.append(test_f3no,ignore_index=True))

train_cluster_index = total_labels[0:train_f3no.shape[0]]
test_cluster_index = total_labels[train_f3no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f3.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f3['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 3 : ',mae_sum/mae_cnt)


#----fold4---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='euclidean',linkage='ward').fit_predict(train_f4no.append(test_f4no,ignore_index=True))

train_cluster_index = total_labels[0:train_f4no.shape[0]]
test_cluster_index = total_labels[train_f4no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f4.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f4['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 4 : ',mae_sum/mae_cnt)

#----fold5---------
total_labels = AgglomerativeClustering(n_clusters=10,affinity='euclidean',linkage='ward').fit_predict(train_f5no.append(test_f5no,ignore_index=True))

train_cluster_index = total_labels[0:train_f5no.shape[0]]
test_cluster_index = total_labels[train_f5no.shape[0]:]



#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f5.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f5['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 5 : ',mae_sum/mae_cnt)




Mae Fold 1 :  0.8431583275095874
Mae Fold 2 :  0.8515385811787766
Mae Fold 3 :  0.8627318273155296
Mae Fold 4 :  0.862890701972968
Mae Fold 5 :  0.8773060894180568


In [20]:
#Birch clustering 
#from sklearn doc
from sklearn.cluster import Birch

#fold1
#just fitted model changes
fitted_model = Birch(n_clusters=10,threshold=0.25,compute_labels=True).fit(train_f1)

#train_cluster_index has the cluster numbers for the training data(users)
train_cluster_index = fitted_model.predict(train_f1)
#test_cluster_index has the cluster numbers for the test data(users)
test_cluster_index = fitted_model.predict(test_f1)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
#go through test data
for i in range(1,test_f1.shape[0]+1):
  #for each item J
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      #ignore ratings which is not present in u.data
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    #go through each training user 
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #found user at index k in our cluster
        #get user_id of user at index k 
        user_id = train_f1['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          #if our neighbour in our cluster has rated item J then use in finding average
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    #initialize predicted rating as 3 to minimize mae
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 1 : ',mae_sum/mae_cnt)


#same thing for fold 2 only thing is train_f1 is replaced by train_f2 and test_f1-->test_f2
#repeat for 5 folds
#-------------fold2--------------

fitted_model = Birch(n_clusters=10,threshold=0.25,compute_labels=True).fit(train_f2)
train_cluster_index = fitted_model.predict(train_f2)
test_cluster_index = fitted_model.predict(test_f2)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
for i in range(1,test_f2.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f2['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 2 : ',mae_sum/mae_cnt)


#-----------------fold3--------------------

fitted_model = Birch(n_clusters=10,threshold=0.25,compute_labels=True).fit(train_f3)
train_cluster_index = fitted_model.predict(train_f3)
test_cluster_index = fitted_model.predict(test_f3)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
for i in range(1,test_f3.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f3['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 3 : ',mae_sum/mae_cnt)


#------fold 4--------------

fitted_model = Birch(n_clusters=10,threshold=0.25,compute_labels=True).fit(train_f4)
train_cluster_index = fitted_model.predict(train_f4)
test_cluster_index = fitted_model.predict(test_f4)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
for i in range(1,test_f4.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f4['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 4 : ',mae_sum/mae_cnt)


#----------F5---------

fitted_model = Birch(n_clusters=10,threshold=0.25,compute_labels=True).fit(train_f5)
train_cluster_index = fitted_model.predict(train_f5)
test_cluster_index = fitted_model.predict(test_f5)


#calculate ratings for each test user based on cluster
#user will be given avg rating of his cluster
#then calculate mae between predicted and actual rating present in u.data
mae_sum=0
mae_cnt=0
for i in range(1,test_f5.shape[0]+1):
  for j in range(1,1683):
    if(np.isnan(ratings[i][j])==True):
      continue
    #calculate avg rating for item J in cluster of user I
    avg_rating=0
    avg_cnt=0
    for k in range(len(train_cluster_index)):
      if(test_cluster_index[i-1] == train_cluster_index[k]):
        #get user_id of cluster neighbor
        user_id = train_f5['user_id'][k]
        if(np.isnan(ratings[user_id][j])==False):
          avg_rating+=(ratings[user_id][j])
          avg_cnt+=1
    predicted_rating = 3
    if(avg_cnt>0):
      predicted_rating = avg_rating/avg_cnt
    #check mae
    mae_sum += abs(predicted_rating - ratings[i][j])
    mae_cnt+=1
print('Mae Fold 5 : ',mae_sum/mae_cnt)

Mae Fold 1 :  0.8434641289316437
Mae Fold 2 :  0.8477921690601621
Mae Fold 3 :  0.8617299713450177
Mae Fold 4 :  0.8679531042342076
Mae Fold 5 :  0.8678696667142268


In [ ]:
#Birch clustering
#from sklearn docs
from sklearn.cluster import Birch
#---fold1------
fitted_model = Birch(n_clusters=10,threshold=0.25,compute_labels=True).fit(train_f1no)

train_cluster_index = fitted_model.predict(train_f1no)
test_cluster_index = fitted_model.predict(train_f1no)
